In [1]:
!git clone --branch dev https://github.com/rolysr/medical-knowledge-discoverer

Cloning into 'medical-knowledge-discoverer'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 160 (delta 15), reused 19 (delta 14), pack-reused 136
Receiving objects: 100% (160/160), 1.14 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [2]:
%cd medical-knowledge-discoverer/
!ls

/content/medical-knowledge-discoverer
datasets  main.py  ontology   requirements.txt
LICENSE   models   README.md  utils


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 119.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of es-core-news-sm to determine which version is compatible with other requirements. This co

In [4]:
from pathlib import Path
from utils.anntools import Collection
import csv



ref = Path('./datasets/train')
collection = Collection().load_dir(ref)
print(f"Loaded {len(collection)} sentences for fitting.")

relations = {}

for sentence in collection.sentences:
  for relation in sentence.relations:
      origin = relation.from_phrase
      origin_text = origin.text.lower()
      destination = relation.to_phrase
      destination_text = destination.text.lower()

      relations[
          origin_text, origin.label, destination_text, destination.label
      ] = relation.label

# store relations for training in re_train.csv
with open('re_train.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["input", "output"])
  for relation in relations.keys():
    input = str(relation)
    output = str(relations[relation])
    writer.writerow([input, output])

Loaded 1400 sentences for fitting.


In [5]:
!pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 72.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [6]:
# let's get a dataset
import pandas as pd
from sklearn.model_selection import train_test_split

path = "re_train.csv"
df = pd.read_csv(path)
df.head()

,input,output
0,"('numbers', 'Concept', 'off-diagonal', 'Concept')",in-context
1,"('expression', 'Concept', 'differential', 'Con...",has-property
2,"('two', 'Concept', 'viruses', 'Concept')",in-context
3,"('compared', 'Action', 'mock', 'Concept')",target
4,"('virus', 'Concept', 'third', 'Concept')",has-property


In [7]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"output":"target_text", "input":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = df['source_text']
df

,source_text,target_text
0,"('numbers', 'Concept', 'off-diagonal', 'Concept')",in-context
1,"('expression', 'Concept', 'differential', 'Con...",has-property
2,"('two', 'Concept', 'viruses', 'Concept')",in-context
3,"('compared', 'Action', 'mock', 'Concept')",target
4,"('virus', 'Concept', 'third', 'Concept')",has-property
...,...,...
7715,"('similar', 'Action', 'influenza', 'Concept')",target
7716,"('similar', 'Action', 'characteristics', 'Conc...",subject
7717,"('spread', 'Concept', 'global', 'Concept')",has-property
7718,"('facilitate', 'Action', 'spread', 'Concept')",target


In [8]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_df.shape, test_df.shape

((6948, 2), (772, 2))

In [9]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=3, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [10]:
!ls
!ls ./outputs

datasets  lightning_logs  models    outputs    requirements.txt  utils
LICENSE   main.py	  ontology  README.md  re_train.csv
simplet5-epoch-0-train-loss-0.8293-val-loss-0.5181
simplet5-epoch-1-train-loss-0.5415-val-loss-0.4844
simplet5-epoch-2-train-loss-0.478-val-loss-0.3901


In [11]:
# let's load the trained model for testing:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-0.478-val-loss-0.3901", use_gpu=True)

In [12]:
from utils.anntools import *


test_collection = Collection()
ref = Path('./datasets/test25')
collection = Collection().load_dir(ref)

for k in range(len(collection.sentences)):
  sentence = collection.sentences[k]
  test_sentence = Sentence(sentence.text)
  for keyphrase in sentence.keyphrases:
    test_sentence.keyphrases.append(keyphrase)
  for i in range(len(sentence.keyphrases)):
    for j in range(len(sentence.keyphrases)):
      if i != j:
        print(k, i, j)
        keyphrase1 = test_sentence.keyphrases[i]
        keyphrase2 = test_sentence.keyphrases[j]
        model_input_prediction = (keyphrase1.text, keyphrase1.label, keyphrase2.text, keyphrase2.label)
        prediction_input = str(model_input_prediction)
        test_sentence.relations.append(Relation(sentence, keyphrase1, keyphrase2, model.predict(prediction_input)))
  test_collection.sentences.append(test_sentence)

print(test_collection)

0 0 1
0 0 2
0 0 3
0 1 0
0 1 2
0 1 3
0 2 0
0 2 1
0 2 3
0 3 0
0 3 1
0 3 2
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 3
1 2 4
1 3 0
1 3 1
1 3 2
1 3 4
1 4 0
1 4 1
1 4 2
1 4 3
2 0 1
2 0 2
2 0 3
2 1 0
2 1 2
2 1 3
2 2 0
2 2 1
2 2 3
2 3 0
2 3 1
2 3 2
3 0 1
3 0 2
3 0 3
3 0 4
3 0 5
3 0 6
3 0 7
3 0 8
3 0 9
3 1 0
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 1 7
3 1 8
3 1 9
3 2 0
3 2 1
3 2 3
3 2 4
3 2 5
3 2 6
3 2 7
3 2 8
3 2 9
3 3 0
3 3 1
3 3 2
3 3 4
3 3 5
3 3 6
3 3 7
3 3 8
3 3 9
3 4 0
3 4 1
3 4 2
3 4 3
3 4 5
3 4 6
3 4 7
3 4 8
3 4 9
3 5 0
3 5 1
3 5 2
3 5 3
3 5 4
3 5 6
3 5 7
3 5 8
3 5 9
3 6 0
3 6 1
3 6 2
3 6 3
3 6 4
3 6 5
3 6 7
3 6 8
3 6 9
3 7 0
3 7 1
3 7 2
3 7 3
3 7 4
3 7 5
3 7 6
3 7 8
3 7 9
3 8 0
3 8 1
3 8 2
3 8 3
3 8 4
3 8 5
3 8 6
3 8 7
3 8 9
3 9 0
3 9 1
3 9 2
3 9 3
3 9 4
3 9 5
3 9 6
3 9 7
3 9 8
4 0 1
4 0 2
4 0 3
4 0 4
4 0 5
4 0 6
4 0 7
4 0 8
4 0 9
4 1 0
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 1 9
4 2 0
4 2 1
4 2 3
4 2 4
4 2 5
4 2 6
4 2 7
4 2 8
4 2 9
4 3 0
4 3 1
4 3 2
4 3 4
4 3 5
4 3 

In [66]:
# coding: utf8

import argparse
import re
from typing import List
import warnings
from pathlib import Path

try:
    from utils.anntools import Collection, Keyphrase, Relation
except ModuleNotFoundError:
    from anntools import Collection, Keyphrase, Relation

scenarios = {
        1: ("scenario1-main", True, True),
        2: ("scenario2-taskA", True, False),
        3: ("scenario3-taskB", False, True),
    }

def test_model(input_data: Collection):
    test_collection = Collection()

    for k in range(len(input_data.sentences)):
      sentence = input_data.sentences[k]
      test_sentence = Sentence(sentence.text)
      for keyphrase in sentence.keyphrases:
        test_sentence.keyphrases.append(keyphrase)
      for i in range(len(sentence.keyphrases)):
        for j in range(len(sentence.keyphrases)):
          if i != j:
            print(k, i, j)
            keyphrase1 = test_sentence.keyphrases[i]
            keyphrase2 = test_sentence.keyphrases[j]
            model_input_prediction = (keyphrase1.text, keyphrase1.label, keyphrase2.text, keyphrase2.label)
            prediction_input = str(model_input_prediction)
            test_sentence.relations.append(Relation(sentence, keyphrase1.id, keyphrase2.id, model.predict(prediction_input)))
      test_collection.sentences.append(test_sentence)
    return test_collection

def run(collection, taskA, taskB):
    """Its supposed to run the test example"""
    # gold_keyphrases, gold_relations = self.model
    collection = collection.clone()

    if taskA:
        collection = test_model(collection)
    if taskB:
        collection = test_model(collection)
    return collection

def eval(path: Path, scenarios_indexes: List[int], submit: Path, run_num):
        """Function that evals according to the baseline classifier"""
        # Its not changed 
        for id in scenarios_indexes:
            folder, taskA, taskB = scenarios[id]

            scenario = path
            print(f"Evaluating on {scenario}.")

            input_data = Collection().load(scenario / "medline.25.txt")
            print(f"Loaded {len(input_data)} input sentences.")
            output_data = run(input_data, taskA, taskB)

            print(f"Writing output to {submit / folder }")
            output_data.dump(submit / folder  / "output.txt", skip_empty_sentences=False)

In [67]:
test_path = Path('./datasets/test25')
submit_path = Path('./results/t5/test25')
run_num = 'run1'
eval(test_path, [3], submit_path, run_num)

Evaluating on datasets/test25.
Loaded 25 input sentences.
0 0 1
0 0 2
0 0 3
0 1 0
0 1 2
0 1 3
0 2 0
0 2 1
0 2 3
0 3 0
0 3 1
0 3 2
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 2
1 1 3
1 1 4
1 2 0
1 2 1
1 2 3
1 2 4
1 3 0
1 3 1
1 3 2
1 3 4
1 4 0
1 4 1
1 4 2
1 4 3
2 0 1
2 0 2
2 0 3
2 1 0
2 1 2
2 1 3
2 2 0
2 2 1
2 2 3
2 3 0
2 3 1
2 3 2
3 0 1
3 0 2
3 0 3
3 0 4
3 0 5
3 0 6
3 0 7
3 0 8
3 0 9
3 1 0
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 1 7
3 1 8
3 1 9
3 2 0
3 2 1
3 2 3
3 2 4
3 2 5
3 2 6
3 2 7
3 2 8
3 2 9
3 3 0
3 3 1
3 3 2
3 3 4
3 3 5
3 3 6
3 3 7
3 3 8
3 3 9
3 4 0
3 4 1
3 4 2
3 4 3
3 4 5
3 4 6
3 4 7
3 4 8
3 4 9
3 5 0
3 5 1
3 5 2
3 5 3
3 5 4
3 5 6
3 5 7
3 5 8
3 5 9
3 6 0
3 6 1
3 6 2
3 6 3
3 6 4
3 6 5
3 6 7
3 6 8
3 6 9
3 7 0
3 7 1
3 7 2
3 7 3
3 7 4
3 7 5
3 7 6
3 7 8
3 7 9
3 8 0
3 8 1
3 8 2
3 8 3
3 8 4
3 8 5
3 8 6
3 8 7
3 8 9
3 9 0
3 9 1
3 9 2
3 9 3
3 9 4
3 9 5
3 9 6
3 9 7
3 9 8
4 0 1
4 0 2
4 0 3
4 0 4
4 0 5
4 0 6
4 0 7
4 0 8
4 0 9
4 1 0
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 1 9
4 2 0
4 2 1
4 2 3
4 2 4
4 2 5


In [68]:
%ls ./results/t5/test25/scenario3-taskB

output.ann  output.txt


In [69]:
from utils.score import main_scenario, report_main


def main_score(gold: Path, submit: Path, verbose:bool, scenarios_indexes: List[int], runs: List[int], prefix:str):
    runs_data = {}

    for run in runs:
        run_data = {}

        if not (submit / f"run{run}").exists():
            print(f"Run {run} not found!")
            continue

        for id in scenarios_indexes:
            folder, skipA, skipB = scenarios[id]

            print(f"Scoring scenario {id} on run {run}:\n")
            run_data[folder.split("-")[0]] = main_scenario(gold/"medline.25.txt", submit / folder / "output.txt", skipA, skipB, verbose)
            print()

        runs_data[f"run{run}"] = run_data

    print()
    report_main(runs_data, prefix)
    return runs_data

In [70]:
main_score(Path('./datasets/test25'), Path('./results/t5/test25'), None, [3], [1], '')

Scoring scenario 3 on run 1:

correct_A: 166
incorrect_A: 0
partial_A: 0
spurious_A: 0
missing_A: 0
--------------------
recall: 1.0
precision: 1.0
f1: 1.0




scenario1_f1:0
scenario1_precision:0
scenario1_recall:0
scenario1_best:0
scenario2_f1:0
scenario2_precision:0
scenario2_recall:0
scenario2_best:0
scenario3_f1:1.0
scenario3_precision:1.0
scenario3_recall:1.0
scenario3_best:run1


{'run1': {'scenario3': {'recall': 1.0, 'precision': 1.0, 'f1': 1.0}}}